In [1]:
# This notebook is used to find a network from a list of nodes in Translator

In [1]:
from TCT import name_resolver
from TCT import node_normalizer
from TCT import translator_metakg
from TCT import translator_kpinfo
from TCT import translator_query
from TCT import TCT

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import requests
import json
import pandas as pd
import  seaborn as sns
import networkx as nx
import numpy as np
import ipycytoscape
import yaml

In [2]:
def load_translator_resources():
    """
    Load the necessary resources for the Translator.
    """
    Translator_KP_info,APInames= translator_kpinfo.get_translator_kp_info()
    metaKG = translator_metakg.get_KP_metadata(APInames) 
    APInames,metaKG = translator_metakg.add_plover_API(APInames, metaKG)
    return  APInames, metaKG, Translator_KP_info


def visualize_interaction_network(result_json):
    """
    Visualize the interaction network from the result JSON.

    Parameters:
    result_json (dict): The result JSON containing subjects, objects, and predicates.
    
    """
    import pandas as pd
    import networkx as nx
    import ipycytoscape

    subjects = []
    objects = []
    predicates = []

    for k, v in result_json.items():
        subjects.append(v['subject'])
        objects.append(v['object'])
        predicates.append(v['predicate'])

    # Convert subject and object CURIEs to names using lookup
    subject_names = []
    object_names = []

    # Use batch_lookup for all subjects and objects to minimize API calls
    unique_nodes = set(subjects + objects)
    node_info_dict = name_resolver.batch_lookup(list(unique_nodes))
    print(node_info_dict)

    for subj, obj in zip(subjects, objects):
        subj_info = node_info_dict.get(subj)
        obj_info = node_info_dict.get(obj)
        subject_names.append(subj_info.name if subj_info and hasattr(subj_info, 'name') else subj)
        object_names.append(obj_info.name if obj_info and hasattr(obj_info, 'name') else obj)


    df = pd.DataFrame({
        "Subject": subject_names,
        "Object": object_names,
        "Predicate": predicates,
    })

    print(df.head())
    # merge predicates and sources
    # Convert 'Sources' to a string representation
    # Remove duplicate edges
    df = df.drop_duplicates()

    # Build networkx graph
    G1 = nx.from_pandas_edgelist(df, source='Subject', target='Object', edge_attr=['Predicate'], create_using=nx.MultiGraph)

    # Cytoscape style
    graph_style = [
        {'selector': 'node', 'style': {'background-color': 'lightblue', 'shape': 'ellipse', 'label': 'data(id)', 'font-size': '12px'}},
        {'selector': 'edge', 'style': {'label': 'data(Predicate)', 'font-size': '10px', "curve-style": "bezier"}}
    ]

    cyto_graph = ipycytoscape.CytoscapeWidget()
    cyto_graph.graph.add_graph_from_networkx(G1, directed=True)
    cyto_graph.set_style(graph_style)
    cyto_graph.set_layout(name='cose', nodeSpacing=80, edgeLengthVal=50)

    display(cyto_graph)

    return df


In [3]:
APInames, metaKG, Translator_KP_info= load_translator_resources()

All_predicates = list(set(metaKG['Predicate']))
All_categories = list((set(list(set(metaKG['Subject']))+list(set(metaKG['Object'])))))
API_withMetaKG = list(set(metaKG['API']))

    # generate a dictionary of API and its predicates
API_predicates = {}
for api in API_withMetaKG:
    API_predicates[api] = list(set(metaKG[metaKG['API'] == api]['Predicate']))

In [4]:
# input_nodes
input_list = ['NPM1', 'FLT3', 'NRAS', 'BCL2', 'WNT7B']
input_info = name_resolver.batch_lookup(input_list)
input_node1_list = [] # get the curie of the nodes in the input_info dictionary
input_node1_list = [input_info[node].curie for node in input_list]
input_node1_category = []
for node in input_list:
    if hasattr(input_info[node], 'types'):
        input_node1_category = input_node1_category + input_info[node].types
input_node1_category = list(set(input_node1_category))

# query nodes
input_node2_list = []
input_node2_category = ['biolink:Gene', 'biolink:Protein', 'biolink:ChemicalSubstance', 'biolink:Drug', 'biolink:DiseaseOrPhenotypicFeature']

sele_predicates = list(set(TCT.select_concept(sub_list=input_node1_category,obj_list=input_node2_category,metaKG=metaKG)))
print("all relevant predicates in Translator:")
print(sele_predicates)


# Get all APIs for the input node1 and node2, user can furter select the APIs among this list
sele_APIs = TCT.select_API(sub_list=input_node1_category,obj_list=input_node2_category,metaKG=metaKG)
print("all relevant APIs in Translator:")
print(sele_APIs)
print(len(sele_APIs))

#sele_APIs = ['Multiomics BigGIM-DrugResponse KP API']
sele_APIs = sele_APIs
# get API URLs : two options: filter by API names or filter by predicates
API_URLs = TCT.get_Translator_API_URL(sele_APIs, 
                                      APInames)

API_URLs

all relevant predicates in Translator:
['biolink:has_member', 'biolink:transcribed_from', 'biolink:disrupts', 'biolink:is_sequence_variant_of', 'biolink:affects', 'biolink:part_of', 'biolink:interacts_with', 'biolink:directly_physically_interacts_with', 'biolink:composed_primarily_of', 'biolink:biomarker_for', 'biolink:causes', 'biolink:associated_with_resistance_to', 'biolink:increases_response_to', 'biolink:derives_from', 'biolink:similar_to', 'biolink:produces', 'biolink:in_complex_with', 'biolink:contraindicated_in', 'biolink:associated_with', 'biolink:occurs_together_in_literature_with', 'biolink:translates_to', 'biolink:has_input', 'biolink:derives_into', 'biolink:colocalizes_with', 'biolink:has_gene_product', 'biolink:affects_response_to', 'biolink:gene_associated_with_condition', 'biolink:associated_with_increased_likelihood_of', 'biolink:has_phenotype', 'biolink:coexists_with', 'biolink:regulates', 'biolink:GeneAffectsChemicalAssociation', 'biolink:positively_correlated_with',

['https://multiomics.rtx.ai:9990/BigGIM_DrugResponse_PerformancePhase/query',
 'https://automat.transltr.io/monarch-kg/query/',
 'https://multiomics.rtx.ai:9990/multiomics/query',
 'https://bte.transltr.io/v1/team/Service%20Provider/query/',
 'https://multiomics.rtx.ai:9990/PharmacogenomicsKG/query',
 'https://automat.transltr.io/cam-kp/query/',
 'https://automat.transltr.io/robokopkg/query/',
 'https://automat.transltr.io/pharos/query/',
 'https://multiomics.rtx.ai:9990/mbkp/query',
 'https://molepro-trapi.transltr.io/molepro/trapi/v1.5/query/',
 'https://kg2cploverdb.ci.transltr.io/kg2c/query',
 'https://automat.renci.org/ehr-clinical-connections-kp/query/',
 'https://automat.transltr.io/icees-kg/query/',
 'https://chp-api.transltr.io/query/',
 'https://cooccurrence.transltr.io/query/',
 'https://automat.renci.org/ehr-may-treat-kp/query/',
 'https://automat.transltr.io/genome-alliance/query/',
 'https://spokekp.transltr.io/api/v1.5/query/',
 'https://ia.transltr.io/api/v1.5/query/',


In [5]:
query_json = TCT.format_query_json(input_node1_list,  # a list of identifiers for input node1
                                   input_node2_list,  # it can be empty list if only want to query node1
                                   input_node1_category,  # a list of categories of input node1
                                   input_node2_category,  # a list of categories of input node2
                                   sele_predicates) # a list of pre

In [6]:
# Step 5: Query Translator APIs and parse results

result = translator_query.parallel_api_query(query_json=query_json, 
                             select_APIs= sele_APIs, 
                             APInames=APInames,
                             API_predicates=API_predicates,
                             max_workers=len(sele_APIs))

'Multiomics KP - TRAPI 1.5.0' generated an exception: argument of type 'NoneType' is not iterable
Microbiome KP - TRAPI 1.5.0: Success!
'MolePro' generated an exception: argument of type 'NoneType' is not iterable
'imProving Agent for TRAPI 1.5' generated an exception: argument of type 'NoneType' is not iterable
'Automat-icees-kg(Trapi v1.5.0)' generated an exception: argument of type 'NoneType' is not iterable
'SPOKE KP for TRAPI 1.5' generated an exception: argument of type 'NoneType' is not iterable
Automat-genome-alliance(Trapi v1.5.0): Success!
'Automat-ehr-may-treat-kp(Trapi v1.5.0)' generated an exception: argument of type 'NoneType' is not iterable
'Automat-ehr-clinical-connections-kp(Trapi v1.5.0)' generated an exception: argument of type 'NoneType' is not iterable
'Automat-monarchinitiative(Trapi v1.5.0)' generated an exception: argument of type 'NoneType' is not iterable
'Text Mined Cooccurrence API' generated an exception: argument of type 'NoneType' is not iterable
'Connec

In [7]:
# select the recored in the result which is a dictionary if the subject and objects are both in input_node1_list
result_filtered = {k: v for k, v in result.items() if isinstance(v, dict) and
                   v.get('subject') in input_node1_list and
                   v.get('object') in input_node1_list}

len(result_filtered)
result_df =visualize_interaction_network(result_filtered)


{'NCBIGene:2322': TranslatorNode(curie='NCBIGene:100313282', label='MIR2322', types=['biolink:Gene', 'biolink:GeneOrGeneProduct', 'biolink:GenomicEntity', 'biolink:ChemicalEntityOrGeneOrGeneProduct', 'biolink:PhysicalEssence', 'biolink:OntologyClass', 'biolink:BiologicalEntity', 'biolink:ThingWithTaxon', 'biolink:NamedThing', 'biolink:Entity', 'biolink:PhysicalEssenceOrOccurrent', 'biolink:MacromolecularMachineMixin'], synonyms=None, curie_synonyms=None), 'NCBIGene:4893': TranslatorNode(curie='MESH:C568585', label='S-4893 compound', types=['biolink:ChemicalEntity', 'biolink:PhysicalEssence', 'biolink:ChemicalOrDrugOrTreatment', 'biolink:ChemicalEntityOrGeneOrGeneProduct', 'biolink:ChemicalEntityOrProteinOrPolypeptide', 'biolink:NamedThing', 'biolink:Entity', 'biolink:PhysicalEssenceOrOccurrent'], synonyms=None, curie_synonyms=None), 'NCBIGene:4869': TranslatorNode(curie='PUBCHEM.COMPOUND:676542', label='Cl-4869 peak 2', types=['biolink:SmallMolecule', 'biolink:MolecularEntity', 'biolin

CytoscapeWidget(cytoscape_layout={'name': 'cose', 'nodeSpacing': 80, 'edgeLengthVal': 50}, cytoscape_style=[{'…